**[SQL Home Page](https://www.kaggle.com/learn/intro-to-sql)**

---


# Introduction

[Stack Overflow](https://stackoverflow.com/) is a widely beloved question and answer site for technical questions. You'll probably use it yourself as you keep using SQL (or any programming language). 

Their data is publicly available. What cool things do you think it would be useful for?

Here's one idea:
You could set up a service that identifies the Stack Overflow users who have demonstrated expertise with a specific technology by answering related questions about it, so someone could hire those experts for in-depth help.

In this exercise, you'll write the SQL queries that might serve as the foundation for this type of service.

As usual, run the following cell to set up our feedback system before moving on.

In [1]:
# Set up feedback system
from learntools.core import binder
binder.bind(globals())
from learntools.sql.ex6 import *
print("Setup Complete")

Using Kaggle's public dataset BigQuery integration.
Setup Complete


Run the next cell to fetch the `stackoverflow` dataset.

In [2]:
from google.cloud import bigquery

# Create a "Client" object
client = bigquery.Client()

# Construct a reference to the "stackoverflow" dataset
dataset_ref = client.dataset("stackoverflow", project="bigquery-public-data")

# API request - fetch the dataset
dataset = client.get_dataset(dataset_ref)

Using Kaggle's public dataset BigQuery integration.


# Exercises

### 1) Explore the data

Before writing queries or **JOIN** clauses, you'll want to see what tables are available. 

*Hint*: Tab completion is helpful whenever you can't remember a command. Type `client.` and then hit the tab key. Don't forget the period before hitting tab.

In [4]:
# Get a list of available tables 
tables = list(client.list_tables(dataset))
list_of_tables = [table.table_id for table in tables]
# Your code here

# Print your answer
print(list_of_tables)

# Check your answer
q_1.check()

['badges', 'comments', 'post_history', 'post_links', 'posts_answers', 'posts_moderator_nomination', 'posts_orphaned_tag_wiki', 'posts_privilege_wiki', 'posts_questions', 'posts_tag_wiki', 'posts_tag_wiki_excerpt', 'posts_wiki_placeholder', 'stackoverflow_posts', 'tags', 'users', 'votes']


<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

For the solution, uncomment the line below.

In [5]:
#q_1.solution()

### 2) Review relevant tables

If you are interested in people who answer questions on a given topic, the `posts_answers` table is a natural place to look. Run the following cell, and look at the output.

In [6]:
# Construct a reference to the "posts_answers" table
answers_table_ref = dataset_ref.table("posts_answers")

# API request - fetch the table
answers_table = client.get_table(answers_table_ref)

# Preview the first five lines of the "posts_answers" table
client.list_rows(answers_table, max_results=5).to_dataframe()

,id,title,body,accepted_answer_id,answer_count,comment_count,community_owned_date,creation_date,favorite_count,last_activity_date,last_edit_date,last_editor_display_name,last_editor_user_id,owner_display_name,owner_user_id,parent_id,post_type_id,score,tags,view_count
0,63172172,None,<p>Up to now 7 answers. But I am really surpri...,None,None,0,None,2020-07-30 11:26:18.453000+00:00,None,2020-07-30 11:26:18.453000+00:00,None,None,None,None,9666018,63159655,2,0,None,None
1,63172205,None,"<p>I know this is an old one, but here's a bit...",None,None,0,None,2020-07-30 11:28:08.737000+00:00,None,2020-07-30 11:28:08.737000+00:00,None,None,None,None,8452082,51420271,2,0,None,None
2,63172206,None,<p>You could try entering this into your URL b...,None,None,2,None,2020-07-30 11:28:11.363000+00:00,None,2020-07-30 11:28:11.363000+00:00,None,None,None,None,13665232,63169739,2,0,None,None
3,63172232,None,<blockquote>\n<p>Here you get simple spring bo...,None,None,0,None,2020-07-30 11:29:20.073000+00:00,None,2020-07-30 11:29:20.073000+00:00,None,None,None,None,5375134,22502400,2,0,None,None
4,63172234,None,"<pre class=""lang-py prettyprint-override""><cod...",None,None,2,None,2020-07-30 11:29:23.420000+00:00,None,2020-07-30 11:29:23.420000+00:00,None,None,None,None,2066113,63172053,2,0,None,None


It isn't clear yet how to find users who answered questions on any given topic. But `posts_answers` has a `parent_id` column. If you are familiar with the Stack Overflow site, you might figure out that the `parent_id` is the question each post is answering.

Look at `posts_questions` using the cell below.

In [7]:
# Construct a reference to the "posts_questions" table
questions_table_ref = dataset_ref.table("posts_questions")

# API request - fetch the table
questions_table = client.get_table(questions_table_ref)

# Preview the first five lines of the "posts_questions" table
client.list_rows(questions_table, max_results=5).to_dataframe()

,id,title,body,accepted_answer_id,answer_count,comment_count,community_owned_date,creation_date,favorite_count,last_activity_date,last_edit_date,last_editor_display_name,last_editor_user_id,owner_display_name,owner_user_id,parent_id,post_type_id,score,tags,view_count
0,9028004,(android)How can do the package name don't sho...,<p>How can do the package name don't shown on ...,NaN,3,5,None,2012-01-27 01:44:30.933000+00:00,None,2012-01-27 18:26:47.597000+00:00,NaT,None,NaN,None,1332870,None,1,0,android|package,256
1,9043121,What type of activity animation view is this?,"<p>In the picture below, what type of view is ...",9043369.0,4,0,None,2012-01-28 05:15:51.087000+00:00,None,2012-01-28 06:55:57.020000+00:00,NaT,None,NaN,None,1125429,None,1,0,iphone|objective-c|ios|uiview|ios5,256
2,9047570,Checking Internet connectivity on android,<p>I want to display a message for user if he ...,NaN,5,3,None,2012-01-28 18:06:04.300000+00:00,None,2014-10-20 23:04:23.583000+00:00,2014-06-11 14:43:37.330000+00:00,user3727964,NaN,None,1074986,None,1,0,java|android,256
3,9061115,Storing web site configuration,<p>I'm looking for a place to store the config...,9061129.0,3,0,None,2012-01-30 08:23:43.587000+00:00,None,2012-01-30 09:25:08.983000+00:00,2012-01-30 08:42:07.270000+00:00,None,601179.0,None,1012902,None,1,0,asp.net-mvc|asp.net-mvc-3|web-config,256
4,9071340,one link to multiple container anchors,<p>I'm trying to solve a problem I am having w...,NaN,1,0,None,2012-01-30 21:52:48.443000+00:00,None,2012-01-31 13:45:18.073000+00:00,2012-01-31 13:45:18.073000+00:00,None,1071061.0,None,1071061,None,1,0,javascript|jquery|html|css,256


Are there any fields that identify what topic or technology each question is about? If so, how could you find the IDs of users who answered questions about a specific topic?

Think about it, and then check the solution by running the code in the next cell.

In [8]:
# Check your answer (Run this code cell to receive credit!)
q_2.solution()

<IPython.core.display.Javascript object>

<span style="color:#33cc99">Solution:</span> 
`posts_questions` has a column called `tags` which lists the topics/technologies each question is about.

`posts_answers` has a column called `parent_id` which identifies the ID of the question each answer is responding to.
`posts_answers` also has an `owner_user_id` column which specifies the ID of the user who answered the question.

You can join these two tables to:
- determine the `tags` for each answer, and then
- select the `owner_user_id` of the answers on the desired tag.

This is exactly what you will do over the next few questions.


### 3) Selecting the right questions

A lot of this data is text. 

We'll explore one last technique in this course which you can apply to this text.

A **WHERE** clause can limit your results to rows with certain text using the **LIKE** feature. For example, to select just the third row of the `pets` table from the tutorial, we could use the query in the picture below.

![](https://i.imgur.com/RccsXBr.png) 

You can also use `%` as a "wildcard" for any number of characters. So you can also get the third row with:

```
query = """
        SELECT * 
        FROM `bigquery-public-data.pet_records.pets` 
        WHERE Name LIKE '%ipl%'
        """
```

Try this yourself. Write a query that selects the `id`, `title` and `owner_user_id` columns from the `posts_questions` table. 
- Restrict the results to rows that contain the word "bigquery" in the `tags` column. 
- Include rows where there is other text in addition to the word "bigquery" (e.g., if a row has a tag "bigquery-sql", your results should include that too).

In [12]:
# Your code here
questions_query = """
                  SELECT id, title, owner_user_id
                  FROM `bigquery-public-data.stackoverflow.posts_questions`
                  WHERE tags LIKE '%bigquery%'
                  """
                  

# Set up the query (cancel the query if it would use too much of 
# your quota, with the limit set to 1 GB)
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)
questions_query_job = client.query(questions_query, job_config=safe_config) # Your code goes here

# API request - run the query, and return a pandas DataFrame
questions_results = questions_query_job.to_dataframe() # Your code goes here

# Preview results
print(questions_results.head())

# Check your answer
q_3.check()

         id                                              title  owner_user_id
0  20477506  Exceeded maximum execution time on Google App ...      2550621.0
1  20520842  BigQuery for logging events of different types...      3091118.0
2  20624837          Google Big Query with C# Server to Server       292779.0
3  20512847  How can I see the currently reached used quota...      2887318.0
4  20484663   Peak Detection and Event Duration using BigQuery       132438.0


<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

For a hint or the solution, uncomment the appropriate line below.

In [13]:
#q_3.hint()
q_3.solution()

### 4) Your first join
Now that you have a query to select questions on any given topic (in this case, you chose "bigquery"), you can find the answers to those questions with a **JOIN**.  

Write a query that returns the `id`, `body` and `owner_user_id` columns from the `posts_answers` table for answers to "bigquery"-related questions. 
- You should have one row in your results for each answer to a question that has "bigquery" in the tags.  
- Remember you can get the tags for a question from the `tags` column in the `posts_questions` table.

Here's a reminder of what a **JOIN** looked like in the tutorial:
```
query = """
        SELECT p.Name AS Pet_Name, o.Name AS Owner_Name
        FROM `bigquery-public-data.pet_records.pets` as p
        INNER JOIN `bigquery-public-data.pet_records.owners` as o 
            ON p.ID = o.Pet_ID
        """
```

It may be useful to scroll up and review the first several rows of the `posts_answers` and `posts_questions` tables.  

In [16]:
# Your code here
answers_query = """
                SELECT a.id, a.body, a.owner_user_id
                FROM `bigquery-public-data.stackoverflow.posts_questions` AS q 
                INNER JOIN `bigquery-public-data.stackoverflow.posts_answers` AS a
                    ON q.id = a.parent_id
                WHERE q.tags LIKE '%bigquery%'
                """

# Set up the query
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)
answers_query_job = client.query(answers_query, job_config=safe_config) # Your code goes here

# API request - run the query, and return a pandas DataFrame
answers_results = answers_query_job.to_dataframe()
# Your code goes here

# Preview results
print(answers_results.head())

# Check your answer
q_4.check()

         id                                               body  owner_user_id
0  14105121  <p>For what it's worth, we've seen the same is...      1932483.0
1  14207800  <p>I had found that this error occurred if (wh...      1335105.0
2  14227124  <p>I was able to get this to work after testin...      1877519.0
3  14278122  <p>I sorted out the problem:-</p>\n\n<p>The ti...      1756536.0
4  14443521  <pre><code>SELECT Pollen.start, DNase.start, D...      1576196.0


<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

For a hint or the solution, uncomment the appropriate line below.

In [17]:
#q_4.hint()
#q_4.solution()

### 5) Answer the question
You have the merge you need. But you want a list of users who have answered many questions... which requires more work beyond your previous result.

Write a new query that has a single row for each user who answered at least one question with a tag that includes the string "bigquery". Your results should have two columns:
- `user_id` - contains the `owner_user_id` column from the `posts_answers` table
- `number_of_answers` - contains the number of answers the user has written to "bigquery"-related questions

In [19]:
# Your code here
bigquery_experts_query ="""
                         SELECT a.owner_user_id AS user_id, COUNT(1) AS number_of_answers
                         FROM `bigquery-public-data.stackoverflow.posts_questions` AS q
                         INNER JOIN `bigquery-public-data.stackoverflow.posts_answers` AS a
                             ON q.id = a.parent_Id
                         WHERE q.tags LIKE '%bigquery%'
                         GROUP BY a.owner_user_id
                         """

# Set up the query (cancel the query if it would use too much of 
# your quota, with the limit set to 1 GB)
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)
bigquery_experts_query_job = client.query(bigquery_experts_query, job_config=safe_config)

# API request - run the query, and return a pandas DataFrame
bigquery_experts_results = bigquery_experts_query_job.to_dataframe()

# Preview results
print(bigquery_experts_results.head())

# Check your answer
q_5.check()

      user_id  number_of_answers
0  12106431.0                 19
1  13758964.0                  1
2  13693304.0                  1
3   5682185.0                  1
4  13932004.0                  1


<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

For a hint or the solution, uncomment the appropriate line below.

In [18]:
#q_5.hint()
#q_5.solution()

<IPython.core.display.Javascript object>

<span style="color:#33cc99">Solution:</span> 
```python

bigquery_experts_query = """
                         SELECT a.owner_user_id AS user_id, COUNT(1) AS number_of_answers
                         FROM `bigquery-public-data.stackoverflow.posts_questions` AS q
                         INNER JOIN `bigquery-public-data.stackoverflow.posts_answers` AS a
                             ON q.id = a.parent_Id
                         WHERE q.tags LIKE '%bigquery%'
                         GROUP BY a.owner_user_id
                         """

# Set up the query (cancel the query if it would use too much of 
# your quota, with the limit set to 1 GB)
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)
bigquery_experts_query_job = client.query(bigquery_experts_query, job_config=safe_config)

# API request - run the query, and return a pandas DataFrame
bigquery_experts_results = bigquery_experts_query_job.to_dataframe()

```

### 6) Building a more generally useful service

How could you convert what you've done to a general function a website could call on the backend to get experts on any topic?  

Think about it and then check the solution below.

In [ ]:
# Check your answer (Run this code cell to receive credit!)
q_6.solution()

# Congratulations!

You know all the key components to use BigQuery and SQL effectively. Your SQL skills are sufficient to unlock many of the world's largest datasets.

Want to go play with your new powers?  Kaggle has BigQuery datasets available [here](https://www.kaggle.com/datasets?fileType=bigQuery).

---
**[SQL Home Page](https://www.kaggle.com/learn/intro-to-sql)**





*Have questions or comments? Visit the [Learn Discussion forum](https://www.kaggle.com/learn-forum/161314) to chat with other Learners.*